In [2]:
class ParameterValues:
    def __init__(self, epsnx, alfax, betax, epsny, alfay, betay, epsnz, alfaz, betaz):
        self.epsnx = epsnx
        self.alfax = alfax
        self.betax = betax
        self.epsny = epsny
        self.alfay = alfay
        self.betay = betay
        self.epsnz = epsnz
        self.alfaz = alfaz
        self.betaz = betaz

In [3]:
class Particle:
    def __init__(self, Nseed, iq, dt, dW, x, x_prime, y, y_prime):
        self.Nseed = Nseed
        self.iq = iq
        self.dt = dt
        self.dW = dW
        self.x = x
        self.x_prime = x_prime
        self.y = y
        self.y_prime = y_prime
        self.z = 0

    def __repr__(self):
        return f"Particle(Nseed={self.Nseed}, iq={self.iq}, dt={self.dt}, dW={self.dW}, x={self.x}, x'={self.x_prime}, y={self.y}, y'={self.y_prime})"

    def propagate(self, distance):
        import math
        # print(str(self.x) + " " + str(self.y), end=" ")
        # convert angles from milliradians to radians
        angle_x = self.x_prime / 1000 
        angle_y = self.y_prime / 1000
        # print("->", end=" ")
        # calculate new positions
        self.x += distance * math.tan(angle_x)
        self.y += distance * math.tan(angle_y)
        # print(str(self.x) + " " + str(self.y))

import matplotlib.pyplot as plt
import numpy as np
import json
import os

class Hole:
    def __init__(self, center_x, center_y, center_z, diameter):
        self.center_x = center_x
        self.center_y = center_y
        self.center_z = center_z
        self.diameter = diameter

    def __repr__(self):
        return f"Hole(center_x={self.center_x}, center_y={self.center_y}, center_z={self.center_z}, diameter={self.diameter})"
    
class Grid:
    def __init__(self, size_x, size_y, size_z, hole_diameter, separation):
        self.size_x = size_x
        self.size_y = size_y
        self.size_z = size_z
        self.hole_diameter = hole_diameter
        self.separation = separation
        self.holes = self.create_3d_grid()

    def create_3d_grid(self):
        holes = []
        center_x_offset = ((self.size_x) * self.separation) / 2
        center_y_offset = ((self.size_y) * self.separation) / 2
        center_z_offset = ((self.size_z) * self.separation) / 2
        for x in range(self.size_x):
            for y in range(self.size_y):
                for z in range(self.size_z):
                    hole = Hole(center_x=x*self.separation - center_x_offset, 
                                 center_y=y*self.separation - center_y_offset, 
                                 center_z=z*self.separation - center_z_offset, 
                                 diameter=self.hole_diameter)
                    holes.append(hole)
        return holes

    def is_point_in_a_hole(self, point):
        x1, y1, z1 = point
        hole_radius = self.hole_diameter / 2

        # Calculate the indices of the hole the point would belong to if it was in a hole
        x_index = round(x1 / self.separation)
        y_index = round(y1 / self.separation)
        z_index = round(z1 / self.separation)

        # Calculate the center of that hole
        x_center = x_index * self.separation
        y_center = y_index * self.separation
        z_center = z_index * self.separation

        # Check if the point is inside the hole
        # return ((x_center - x1)**2 + (y_center - y1)**2 + (z_center - z1)**2) <= hole_radius**2
        return (x_center - hole_radius <= x1 <= x_center + hole_radius and
                y_center - hole_radius <= y1 <= y_center + hole_radius and
                z_center - hole_radius <= z1 <= z_center + hole_radius)

class dataUtilities:
    def totalParticleFromCoordOut(coord_dot_out_file_location):
        data = []
        with open(coord_dot_out_file_location, 'r') as file:
            next(file)  # Skip the header line
            for line in file:
                line = line.strip()
                if line:
                    values = line.split()
                    particle = Particle(
                        Nseed=int(values[0]),
                        iq=int(values[1]),
                        dt=float(values[2]),
                        dW=float(values[3]),
                        x=float(values[4]),
                        x_prime=float(values[5]),
                        y=float(values[6]),
                        y_prime=float(values[7])
                    )
                    data.append(particle)
        return data

    def particlesInHolesfromGridAndParticles(grid, particle_list):
        particles_in_holes = []
        for particle in particle_list:
            point = (particle.x, particle.y, particle.z)
            if grid.is_point_in_a_hole(point):
                particles_in_holes.append(particle)

        for particle in particles_in_holes:
            particle.propagate(10)

        return particles_in_holes
    
    def imageFromParticlesInHoles(particle_list, filename="heatmap.png"):
        bins_hist = 200

        # don't change these unless underlying data changed
        size_x = 40
        separation = 0.3

        x_in_holes = [particle.x for particle in particle_list]
        y_in_holes = [particle.y for particle in particle_list]

        histo_width = ((size_x * separation) / 2) - 2
        edges_hist = np.linspace(-histo_width, histo_width, bins_hist + 1)
        H, _, _ = np.histogram2d(x_in_holes, y_in_holes, bins=[edges_hist, edges_hist])
        fig, ax = plt.subplots(figsize=(8, 8))
        pcm = ax.pcolormesh(edges_hist, edges_hist, H.T, cmap='Greys') #or use "Greys"

        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_frame_on(False)
        plt.axis('square')

        plt.savefig(filename, dpi=64, bbox_inches='tight', pad_inches=0)
        plt.close(fig)

    def particleListFromJSON(json_data):
        """
            JSON and CSV caches of particles are already preprocessed and have been reduced from coord.dat files,
            which are the complete output from the beamline simulation. These caches have already been passed through a
            pepperpot grid (to reduce the number of particles) and have already been propogated.
            Images can be rapidly generated from them, in comparison to coord.out files.
        """
        particles = json_data[1:]  # the first row is the header

        particle_list = []
        for particle_data in particles:
            particle = Particle(
                Nseed=int(particle_data[0]),
                iq=int(particle_data[1]),
                dt=float(particle_data[2]),
                dW=float(particle_data[3]),
                x=float(particle_data[4]),
                x_prime=float(particle_data[5]),
                y=float(particle_data[6]),
                y_prime=float(particle_data[7])
            )
            particle_list.append(particle)

        return particle_list
    
    def particleListToJSON(particles_in_holes):
        csv_data = []
        csv_data.append(["Nseed", "iq", "dt", "dW", "x", "x_prime", "y", "y_prime"])
        for particle in particles_in_holes:
            csv_data.append([particle.Nseed, particle.iq, particle.dt, particle.dW, particle.x, particle.x_prime, particle.y, particle.y_prime])
        return json.dumps(csv_data)



In [3]:
# # write data files names to text file
# import os
# def write_folder_names_to_file(data_folder, cache_file):
#     # Open the cache file in append mode
#     with open(cache_file, "w") as cache:
#         # Walk through the directory
#         for _, dirs, _ in os.walk(data_folder):
#             # For each directory in the directory
#             for dir in dirs:
#                 # Write the directory name to the cache file
#                 cache.write(dir + "\n")

# data_folder = "E:\\data"
# cache_file = "tested_params.txt"
# write_folder_names_to_file(data_folder, cache_file)


In [ ]:
# csv and image generation
import os
import re
import matplotlib.pyplot as plt
import numpy as np
import csv
import json

subfolder = "D:/Ian-Data"
os.makedirs("images", exist_ok=True)
os.makedirs("csv", exist_ok=True)
i = 0

for foldername in os.listdir(subfolder):
    print(i)
    i += 1
    folder_path = os.path.join(subfolder, foldername)
    csv_path = os.path.join(folder_path, f"data.json")

    # JSON
    if os.path.exists(csv_path):
        print(f"Image data.json already exist, skipping...")
        continue

    if os.path.isdir(folder_path):
        coord_file = os.path.join(folder_path, "coord.out")
        if os.path.isfile(coord_file):
            data = []
            print(foldername)
            with open(coord_file, 'r') as file:
                next(file)
                for line in file:
                    line = line.strip()
                    if line:
                        values = line.split()
                        particle = Particle(
                            Nseed=int(values[0]),
                            iq=int(values[1]),
                            dt=float(values[2]),
                            dW=float(values[3]),
                            x=float(values[4]),
                            x_prime=float(values[5]),
                            y=float(values[6]),
                            y_prime=float(values[7])
                        )
                        data.append(particle)

            size_x, size_y, size_z = 40, 40, 1
            hole_diameter = 0.01
            separation = 0.3

            grid = Grid(size_x, size_y, size_z, hole_diameter, separation)

            particles_in_holes = []
            for particle in data:
                point = (particle.x, particle.y, particle.z)
                if grid.is_point_in_a_hole(point):
                    particles_in_holes.append(particle)

            for particle in particles_in_holes:
                particle.propagate(10)

            if particles_in_holes:
                x_in_holes = [particle.x for particle in particles_in_holes]
                y_in_holes = [particle.y for particle in particles_in_holes]

            bins_hist = 200

            # don't change these unless underlying data changed
            size_x = 40
            separation = 0.3

            histo_width = ((size_x * separation) / 2) - 2
            edges_hist = np.linspace(-histo_width, histo_width, bins_hist + 1)
            H, _, _ = np.histogram2d(x_in_holes, y_in_holes, bins=[edges_hist, edges_hist])
            fig, ax = plt.subplots(figsize=(8, 8))
            pcm = ax.pcolormesh(edges_hist, edges_hist, H.T, cmap='Greys') #or use "Greys"

            ax.set_xticks([])
            ax.set_yticks([])
            ax.set_frame_on(False)

            plt.axis('square')

            plt.savefig(f"{folder_path}/{foldername}.png", dpi=64, bbox_inches='tight', pad_inches=0)

            plt.close(fig)

            # Save particle data to CSV
            csv_data = []
            csv_data.append(["Nseed", "iq", "dt", "dW", "x", "x_prime", "y", "y_prime"])
            for particle in particles_in_holes:
                csv_data.append([particle.Nseed, particle.iq, particle.dt, particle.dW, particle.x, particle.x_prime, particle.y, particle.y_prime])

            # Convert csv_data to a JSON string
            csv_data_json = json.dumps(csv_data)

            csv_json_path = os.path.join(folder_path, f"data.json")
            with open(csv_json_path, 'w') as json_file:
                json_file.write(csv_data_json)

    print("CSV generated successfully.")



In [2]:
import os

# define the directory you want to start from
start_directory = "E://data/"

for dirpath, dirnames, filenames in os.walk(start_directory):
    if not 'data.json' in filenames:
        print(f"'data.json' not found in: {os.path.abspath(dirpath)}")


'data.json' not found in: E:\data


In [4]:
# generate images from local JSONs
import os
import json
import matplotlib.pyplot as plt
import numpy as np
import cv2

# Define the local folder path
local_folder_path = 'E://data'

# Define the folder to store the generated images
image_folder = 'local-images-50cm-propagation-bw-200bin-cropped'

# Create the image folder if it doesn't exist
os.makedirs(image_folder, exist_ok=True)

# Iterate over the folders in the local folder
for folder_name in os.listdir(local_folder_path):
    folder_path = os.path.join(local_folder_path, folder_name)

    print(f"Processing folder: {folder_name}")

    # Get the path to the JSON file in the current folder
    json_file_path = os.path.join(folder_path, 'data.json')

    # Check if the JSON file exists
    if not os.path.exists(json_file_path):
        print(f"JSON file not found in {folder_name}. Skipping...")
        continue

    # Generate the image filename
    image_filename = f"{folder_name}.png"
    image_path = os.path.join(image_folder, image_filename)

    if os.path.exists(image_path):
        print("Already exists")
        continue

    # Read the JSON file
    with open(json_file_path, 'r') as json_file:
        json_data = json.load(json_file)

    # Convert JSON data to particle list
    particles_in_holes = dataUtilities.particleListFromJSON(json_data)

    for particle in particles_in_holes:
        particle.propagate(40)

    dataUtilities.imageFromParticlesInHoles(particles_in_holes, filename=image_path)

print("Processing completed.")


Processing folder: epsnx0.05334_alfax2.64713_betax234.57084_epsny0.1_alfay-0.55_betay170.0_epsnz5.0_alfaz0.1_betaz10.0
Processing folder: epsnx0.1_alfax2.15189_betax54.52866_epsny0.1_alfay-0.55_betay170.0_epsnz5.0_alfaz0.1_betaz10.0
Processing folder: epsnx0.1_alfax0.44883_betax368.94668_epsny0.1_alfay-0.55_betay170.0_epsnz5.0_alfaz0.1_betaz10.0
Processing folder: epsnx0.1_alfax-0.76345_betax69.78894_epsny0.1_alfay-0.55_betay170.0_epsnz5.0_alfaz0.1_betaz10.0
Processing folder: epsnx0.03552_alfax4.98674_betax184.34541_epsny0.21656_alfay1.83719_betay191.07403_epsnz5.00000_alfaz0.10000_betaz10.00000
Processing folder: epsnx0.1_alfax1.02763_betax264.12179_epsny0.1_alfay-0.55_betay170.0_epsnz5.0_alfaz0.1_betaz10.0
Processing folder: epsnx0.1_alfax1.45894_betax445.78467_epsny0.1_alfay-0.55_betay170.0_epsnz5.0_alfaz0.1_betaz10.0
Processing folder: epsnx0.04518_alfax2.71813_betax210.79302_epsny0.1_alfay-0.55_betay170.0_epsnz5.0_alfaz0.1_betaz10.0
Processing folder: epsnx0.04914_alfax3.69649_be

In [5]:
# generate images from local JSONs (side-by-sides)
import os
import json
import matplotlib.pyplot as plt
import numpy as np
import cv2
import gc

# Define the local folder path
local_folder_path = 'E://Ian-Data/'

# Define the folder to store the generated images
image_folder = 'local-images-before-after-propagation-bw'

# Create the image folder if it doesn't exist
os.makedirs(image_folder, exist_ok=True)

i = 0
# Iterate over the folders in the local folder
for folder_name in os.listdir(local_folder_path):
    folder_path = os.path.join(local_folder_path, folder_name)
    i += 1

    # Get the path to the JSON file in the current folder
    json_file_path = os.path.join(folder_path, 'data.json')
    print(i)

    # Check if the JSON file exists
    if not os.path.exists(json_file_path):
        print(f"JSON file not found in {folder_name}. Skipping...")
        continue

    # Generate the image filename
    image_filename_one = f"{folder_name}_one.png"
    image_filename_two = f"{folder_name}_two.png"
    image_filename_both = f"{folder_name}.png"

    image_path_one = os.path.join(image_folder, image_filename_one)
    image_path_two = os.path.join(image_folder, image_filename_two)
    image_path_both = os.path.join(image_folder, image_filename_both)

    if os.path.exists(image_path_both):
        # print("Already exists")
        continue

    # Read the JSON file
    with open(json_file_path, 'r') as json_file:
        json_data = json.load(json_file)

    # Convert JSON data to particle list
    particles_in_holes = dataUtilities.particleListFromJSON(json_data)

    # Clear the json_data variable to free memory
    del json_data
    gc.collect()

    dataUtilities.imageFromParticlesInHoles(particles_in_holes, filename=image_path_one)

    # Perform propagation on particles
    for particle in particles_in_holes:
        particle.propagate(40)

    dataUtilities.imageFromParticlesInHoles(particles_in_holes, filename=image_path_two)

    # Clear the particles_in_holes variable to free memory
    del particles_in_holes
    gc.collect()

    # Read the images
    image1 = cv2.imread(image_path_one)
    image2 = cv2.imread(image_path_two)

    # Concatenate the images along the horizontal axis
    side_by_side_image = cv2.hconcat([image1, image2])

    # Clear the image1 and image2 variables to free memory
    del image1, image2
    gc.collect()

    # Save the concatenated images
    cv2.imwrite(image_path_both, side_by_side_image)

    # Clear the side_by_side_image variable to free memory
    del side_by_side_image
    gc.collect()

    os.remove(image_path_one)
    os.remove(image_path_two)
print("Processing completed.")


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
JSON file not found in epsnx0.03620_alfax4.84057_betax301.76044_epsny0.09383_alfay-1.77080_betay276.64272_epsnz5.00000_alfaz0.10000_betaz10.00000. Skipping...
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
2

In [ ]:
# # search data folder for empty coord files

# import os
# import shutil

# # Specify the path to the folder
# path = "E:/data"

# # Iterate over all subdirectories in the specified directory
# for subdir, dirs, files in os.walk(path):
#     # Iterate over all files in the current subdirectory
#     for file in files:
#         # If the file is 'coord.out'
#         if file == 'coord.out':
#             # Get the full path to the file
#             file_path = os.path.join(subdir, file)
#             # Get the size of the file in kilobytes
#             file_size_kb = os.path.getsize(file_path) / 1024
#             # If the size of the file is less than 100 KB
#             if file_size_kb < 100:
#                 # Print the file path and size
#                 print(f"File '{file_path}' is {file_size_kb} KB.")
#                 # Delete the parent folder and all of its contents
#                 # shutil.rmtree(subdir)
#                 # print(f"Parent folder '{subdir}' has been deleted.")
#                 break


In [ ]:
# # delete database duplicates
# import mysql.connector

# # Connect to the database
# def connect_to_database():
#     return mysql.connector.connect(
#         host='srv395.hstgr.io',
#         port='3306',
#         user='u641848469_iank',
#         password='VPCBwkO68!!nCOXa2Yv4',
#         database='u641848469_argonneimages',
#         connect_timeout=10  # Set connection timeout to 10 minutes
# )

# # Function to check for duplicates and delete all except one
# def delete_duplicates():
#     # Connect to the database
#     cnx = connect_to_database()
#     cursor = cnx.cursor()

#     # Query to select duplicate paramsname values and their counts
#     sql_query = "SELECT paramsname, COUNT(*) FROM imagesndata GROUP BY paramsname HAVING COUNT(*) > 1"

#     # Execute the query
#     cursor.execute(sql_query)

#     # Fetch the results
#     results = cursor.fetchall()

#     # Delete duplicates except one
#     for result in results:
#         paramsname, count = result
#         # Determine the number of duplicates to be deleted
#         num_duplicates = count - 1
#         # Delete the duplicates
#         delete_query = "DELETE FROM imagesndata WHERE paramsname = %s LIMIT %s"
#         cursor.execute(delete_query, (paramsname, num_duplicates))
#         # Commit the changes to the database
#         cnx.commit()
#         print(f"{num_duplicates} duplicates deleted for paramsname: {paramsname}")

#     # Close the cursor and connection
#     cursor.close()
#     cnx.close()

# # Call the function to check for duplicates and delete them
# delete_duplicates()


In [8]:
import cv2
import os

# Directory containing images
image_dir = "local-images-before-after-propagation-bw-shorter"

# Output directory
output_dir = 'local-images-before-after-propagation-2x-dataset'

# Create output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Iterate over image files
for file_name in os.listdir(image_dir):
    # Check if the file is an image
    if not file_name.lower().endswith('.png'):
        continue

    # Full path of the current image file
    img_path = os.path.join(image_dir, file_name)

    # Read image in grayscale mode
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    
    # Split image in half
    mid_x = img.shape[1] // 2
    left_img = img[:, :mid_x]
    right_img = img[:, mid_x:]
    
    # Generate output file paths
    base_name = file_name.rsplit('.', 1)[0]
    left_img_path = os.path.join(output_dir, base_name + "_1.png")
    right_img_path = os.path.join(output_dir, base_name + "_2.png")
    
    # Save each half into separate files
    cv2.imwrite(left_img_path, left_img)
    cv2.imwrite(right_img_path, right_img)
